In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import boto3
import json

In [2]:
Target_year = 2023

pd.set_option('display.max_rows', 20)
pd.options.display.float_format = '{:,.1f}'.format

User_Name = 'dsmapadmin'


AWS_folder = "./"

src_folder = os.path.join(AWS_folder,'Downloaded',User_Name)
out_folder = os.path.join(AWS_folder,'ToUpload',User_Name)

In [3]:
# Create Out folder if not exists
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [4]:
# s3 = boto3.client('s3')
# content_object = s3.get_object(Bucket = 'dsmap-photo', Key = 'public/'+User_Name + '/photo.json')
# file_content = content_object['Body'].read().decode('utf-8')
# json_content = json.loads(file_content)

In [10]:
s3_client = boto3.client("s3")

response = s3_client.list_objects_v2(Bucket='dsmap-photo',Prefix="public/MGC002/rgb")
files = response.get("Contents")
for file in files:
    print(f"file_name: {file['Key']}, size: {file['Size']}")

file_name: public/MGC002/rgb/2023090702381912863642494443566783027781808340000000.jpg, size: 7782400
file_name: public/MGC002/rgb/2023090702382112863641405563566782905561807940000000.jpg, size: 10514432
file_name: public/MGC002/rgb/2023090702382512863640600003566782586111803580000000.jpg, size: 10620928
file_name: public/MGC002/rgb/2023090702382712863639888893566782794441802630000000.jpg, size: 10608640
file_name: public/MGC002/rgb/2023090702403112863643708333566785025001804010000000.jpg, size: 8564736
file_name: public/MGC002/rgb/2023090702553712863655372223566566927782147010000000.jpg, size: 10657792
file_name: public/MGC002/rgb/2023090702554712863654738893566567030562147080000000.jpg, size: 10891264
file_name: public/MGC002/rgb/2023090706061612864475211113566296427781570130000000.jpg, size: 11415552
file_name: public/MGC002/rgb/2023090706061812864474977783566296377781569800000000.jpg, size: 11513856
file_name: public/MGC002/rgb/2023090706064012864469938893566302347221570790000000.jp

In [13]:
file['Key']

'public/MGC002/rgb/2023090702381912863642494443566783027781808340000000.jpg'

In [28]:
from PIL import Image
from io import BytesIO

bucket = 'dsmap-photo'

s3 = boto3.client("s3")

for file in files:
  print(f"file_name: {file['Key']}, size: {file['Size']}")
  new_obj = s3.get_object(Bucket=bucket, Key=file['Key'])
  image_dl = new_obj['Body'].read()
  image = Image.open(BytesIO(image_dl))

  print(image.width, image.height)

  image = image.resize((1320,989))
  print(image.width, image.height)

  in_mem_file = BytesIO()
  image.save(in_mem_file, format='jpeg')
  in_mem_file.seek(0)

  # Upload image to s3
  s3.upload_fileobj(
      in_mem_file, # This is what i am trying to upload
      bucket,
      file['Key']
  )

file_name: public/MGC002/rgb/2023090702381912863642494443566783027781808340000000.jpg, size: 7782400
1320 989
1320 989
file_name: public/MGC002/rgb/2023090702382112863641405563566782905561807940000000.jpg, size: 10514432
5280 3956
1320 989
file_name: public/MGC002/rgb/2023090702382512863640600003566782586111803580000000.jpg, size: 10620928
5280 3956
1320 989
file_name: public/MGC002/rgb/2023090702382712863639888893566782794441802630000000.jpg, size: 10608640
5280 3956
1320 989
file_name: public/MGC002/rgb/2023090702403112863643708333566785025001804010000000.jpg, size: 8564736
5280 3956
1320 989
file_name: public/MGC002/rgb/2023090702553712863655372223566566927782147010000000.jpg, size: 10657792
5280 3956
1320 989
file_name: public/MGC002/rgb/2023090702554712863654738893566567030562147080000000.jpg, size: 10891264
5280 3956
1320 989
file_name: public/MGC002/rgb/2023090706061612864475211113566296427781570130000000.jpg, size: 11415552
5280 3956
1320 989
file_name: public/MGC002/rgb/202309

In [24]:
print(image.format)

None


In [12]:
file_to_save = os.path.join(src_folder ,'photo.json')
with open(file_to_save, "w", encoding="utf-8") as file:
  file.write(json.dumps(json_content, ensure_ascii=False))